In [1]:
import os
import numpy as np
import pandas as pd
import h5py
import pickle as pkl
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm

import sys

sys.path.append('..')

import datasets

In [2]:
feats = pd.read_csv('../local/ExportPCS.csv')
feats = feats[feats['did'] == 1]
feats.shape

(14242, 12)

In [3]:
ts_orig = pd.read_csv('../data/Yearly-train.csv').drop(columns='V1')

In [4]:
data = []

for s in tqdm(ts_orig.values):
    ser = pd.Series(s)
    ser = datasets.get_last_N(ser, 24)
    data.append(ser)

100%|██████████| 23000/23000 [00:04<00:00, 4895.96it/s]


In [5]:
data = np.vstack(data)

sc = MinMaxScaler()
data = sc.fit_transform(data.T).T

data.shape

(23000, 24)

In [6]:
data = data[feats['id'].values.astype(int)]
data.shape

(14242, 24)

In [8]:
ind = np.random.permutation(len(data))
ind.shape

(14242,)

In [9]:
test_size = 5000

In [10]:
data_dir = Path('../data')
filename = 'yearly_24_nw'

In [12]:
train_ind = ind[:-test_size].astype(int)
test_ind = ind[-test_size:].astype(int)

with h5py.File(str(data_dir / (filename + '_train.h5')), 'w') as hf:
    hf.create_dataset('X', data=data[train_ind, :-6])
    hf.create_dataset('y', data=data[train_ind, -6:])

with h5py.File(str(data_dir / (filename + '_test.h5')), 'w') as hf:
    hf.create_dataset('X', data=data[test_ind, :-6])
    hf.create_dataset('y', data=data[test_ind, -6:])

with h5py.File(str(data_dir / (filename + '_feats_train.h5')), 'w') as hf:
    hf.create_dataset('X', data=feats.iloc[train_ind, :10])

with h5py.File(str(data_dir / (filename + '_feats_test.h5')), 'w') as hf:
    hf.create_dataset('X', data=feats.iloc[test_ind, :10])